In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor

In [2]:
dfValue = pd.read_csv('train.csv',engine='c',memory_map=True)

In [3]:
dfTest = pd.read_csv('test.csv',engine='c',memory_map=True)

In [4]:
X = dfValue.loc[:, dfValue.columns != 'trip_duration']
Y = dfValue.trip_duration

In [5]:
pickupDatetime  = pd.to_datetime(X.pickup_datetime)
dropoffDatetime = pd.to_datetime(X.dropoff_datetime)

XDrop = X.drop(['id','pickup_datetime','dropoff_datetime'],axis=1)

# On reccréé des colonnes de pickup en séparant les informations de la date
XDrop['pickupHour'] = pickupDatetime.dt.hour
XDrop['pickupDayOfWeek'] = pickupDatetime.dt.dayofweek
XDrop['pickupWeekOfYear'] = pickupDatetime.dt.weekofyear
XDrop['pickupDayOfYear'] = pickupDatetime.dt.dayofyear

# Pareil pour le dropoff
XDrop['dropoffHour'] = dropoffDatetime.dt.hour
XDrop['dropoffDayOfWeek'] = dropoffDatetime.dt.dayofweek
XDrop['dropoffWeekOfYear'] = dropoffDatetime.dt.weekofyear
XDrop['dropoffDayOfYear'] = dropoffDatetime.dt.dayofyear

XTrain, XTest, YTrain, Ytest = train_test_split(XDrop,Y,train_size=0.85,random_state=19061996)

In [6]:
#pipeline = Pipeline(steps=[('preprocessor', preprocessor),('model', model)]).fit(XTrain,YTrain)


## PREPROCESSING LABEL ENCODER ##
# labelEncoder = LabelEncoder()
# XTrainFinal = XTrain.copy()
# XTestFinal = XTest.copy()

# XTrainFinal.store_and_fwd_flag = labelEncoder.fit_transform(XTrain.store_and_fwd_flag)
# XTestFinal.store_and_fwd_flag = labelEncoder.transform(XTest.store_and_fwd_flag)

## PREPROCESSING ONE HOT ENCODER
OHEncoder = OneHotEncoder(sparse=False)
OHColumnsTrain = pd.DataFrame(OHEncoder.fit_transform(XTrain[['store_and_fwd_flag']]))
OHColumnsTest  = pd.DataFrame(OHEncoder.transform(XTest[['store_and_fwd_flag']]))

OHColumnsTrain.index = XTrain.index
OHColumnsTest.index = XTest.index

XTrainCategoricalRemoved = XTrain.loc[:, XTrain.columns != 'store_and_fwd_flag']
XTestCategoricalRemoved = XTest.loc[:, XTest.columns != 'store_and_fwd_flag']

XTrainFinal = pd.concat([XTrainCategoricalRemoved, OHColumnsTrain], axis=1)
XTestFinal = pd.concat([XTestCategoricalRemoved, OHColumnsTest], axis=1)

In [7]:
#n_estimators=150 max_depth=10 --> meilleur résultats pour 10_000 lignes avec OHEncoder
def train (n, d,jobs=None):
    startingMsg = "Starting n={} d={} ".format(n,d)
    print(startingMsg,end='')
    
    model = RandomForestRegressor(random_state=19061996,n_estimators=n, max_depth=d,n_jobs=jobs)
    start = time.time()
    model.fit(XTrainFinal,YTrain)
    end = time.time()
    preds = model.predict(XTestFinal)
    mean = mean_absolute_error(Ytest,preds)

    endMsg = "t={}s e={}\n".format(format(end-start),mean)
    print(endMsg,end='')

In [14]:
N = [70,80,90,100]
D = [30,40,50]

In [15]:
# +---------> D
# |
#\|A/
# N
mask = [[1,1,1],[1,1,1],[1,1,1],[1,1,1]]

#Je vérifie que je n'est pas fait de la merde
assert(len(mask) == len(N))
for i in range(len(N)):
    assert(len(mask[i]) == len(D))

In [16]:
import threading # Les entraînements commencent à être long, il est temps de passer aux threads
lock = threading.Lock()
def threadFunc():
    for i in range(len(N)):
        for j in range (len(D)):
            if mask[i][j] == 1:
                train(N[i],D[j],4)

In [11]:
print(XTrainFinal.shape, ' ', XTestFinal.shape)
threadNum = 3
threads = []
threadFunc()

(1239847, 16)   (218797, 16)
Starting n=50 d=60t=394.1654534339905s e=359.46603197828307
Starting n=60 d=40t=467.8542101383209s e=356.2934803112503
Starting n=60 d=50t=465.10663771629333s e=356.30776700537615
Starting n=60 d=60t=465.98942041397095s e=356.20533701068854
